In [1]:
!python -m ensurepip --upgrade
!python -m pip install --upgrade pip
!pip cache purge # Clear the pip package cache. This can be useful when pip is having issues resolving package dependencies.
!pip install opencv-python
!pip install mediapipe
!pip install os

/usr/bin/python3: No module named ensurepip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Files removed: 6 (1.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 66.5 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [2]:
!pip install csv

ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


In [ ]:
import mediapipe as mp
import cv2
import csv
import os

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose

# Input images and their pose labels
images = {
    '/home/server-iss-mbkm/project/project_student_detection/student_detection_dataset/class_object/tertidur/person_5.jpg': "Tertidur",
    '/home/server-iss-mbkm/project/project_student_detection/student_detection_dataset/class_object/belajar/person_3.jpg': "Belajar",
    '/home/server-iss-mbkm/project/project_student_detection/student_detection_dataset/class_object/memperhatikan/person_0.jpg': "Memperhatikan",
}

# Output CSV file
csv_file_path = "/home/server-iss-mbkm/project/project_student_detection/student_detection_dataset/class_object/"

# Extract landmarks and save to CSV
with open(csv_file_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    # Write header
    header = ["Pose"] + [f"{axis}_{idx}" for idx in range(33) for axis in ["X", "Y", "Z"]]
    writer.writerow(header)

    with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
        for image_path, label in images.items():
            # Read the image
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error reading {image_path}")
                continue

            # Convert to RGB
            results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            # Check if landmarks are detected
            if results.pose_world_landmarks:
                # Extract landmarks
                row = [label]
                for landmark in results.pose_world_landmarks.landmark:
                    row.extend([landmark.x, landmark.y, landmark.z])
                writer.writerow(row)
                print(f"Processed {image_path} - Pose: {label}")
                print(
                    f'NOSE coordinates: ('
                    f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]}'
                    f'LEFT_SHOULDER coordinates: ('
                    f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]}'
                    f'RIGHT_SHOULDER coordinates: ('
                    f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]}'
                )
            else:
                print(f"No landmarks detected in {image_path}")

: 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load CSV data
data = pd.read_csv("/content/drive/My Drive/desertasi/person.csv")

# Separate features and labels
X = data.drop("Pose", axis=1)  # Landmark features
y = data["Pose"]              # Pose labels

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Evaluate the classifier
y_pred = clf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")

Classification Report:
              precision    recall  f1-score   support

     Belajar       0.00      0.00      0.00       0.0
    Tertidur       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0

Accuracy: 0.00


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

In [ ]:
def classify_image(image_path, pose_model, classifier):
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error reading {image_path}")
        return

    # Convert to RGB and process with MediaPipe Pose
    results = pose_model.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Check if landmarks are detected

    if results.pose_world_landmarks:
        # Extract landmarks
        landmarks = []
        for landmark in results.pose_world_landmarks.landmark:
            landmarks.extend([landmark.x, landmark.y, landmark.z])

        # Predict the pose
        landmarks = [landmarks]  # Reshape for the model
        pose_prediction = classifier.predict(landmarks)[0]
        print(landmarks)
        # Display the result
        cv2.putText(image, pose_prediction, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        print(f"Pose Prediction for {image_path}: {pose_prediction}")
    elif results.pose_landmarks == None:
        print(f"Pose Prediction for {image_path}: Tertidur")
    else:
        print(f"No landmarks detected in {image_path}")

# Load the trained classifier and MediaPipe Pose
with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
    classify_image("/content/drive/My Drive/desertasi/pose_objects3/person_11.jpg", pose, clf)

Pose Prediction for /content/drive/My Drive/desertasi/pose_objects3/person_11.jpg: Tertidur
